In [2]:
import math
import numpy as np
import os
import pandas as pd
import subprocess

"""
Correcting for multiple testing of ANOVA p.values
We restrict to eSTRs exclusively because we only care  
to compare between eSTRs and close by eSNPs

"""
DATADIR = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'
SHORTEN = {
    "Artery-Aorta":"Artery.A"     ,
    "Artery-Tibial": "Artery.T",
    "Adipose-Subcutaneous": "Adipose.S",    
    "Adipose-Visceral":"Adipose.V",
    "Brain-Caudate(basalganglia)":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "Mucosa",
    "Esophagus-Muscularis":"Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve.T",
    "Skin-NotSunExposed(Suprapubic)": "SkinUnexposed",
    "Skin-SunExposed(Lowerleg)":"SkinLeg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}
tissues = sorted([item for item in list(SHORTEN.keys())])

In [11]:
T = tissues[0]
linreg = pd.read_csv(os.path.join(DATADIR, T, "PQValues"), sep="\t")
estrs = linreg.loc[linreg['qvalue']<0.1].copy()

anova=pd.read_csv(os.path.join(DATADIR, T,'HH/anova_wg.csv') )

to_correct = anova.loc[anova['gene'].isin(list(estrs['gene']))].copy()
print(linreg.shape,estrs.shape, anova.shape, to_correct.shape)

(87891, 12) (1789, 12) (4424, 13) (3615, 13)


In [21]:
#Save pval in file and FDR correct
to_correct['anova_pval'].to_csv('pvalues.txt', sep='\n', index=False)
Tell = subprocess.call("./fdr_correct.r") #copied the file here

#FDR corrected... add to dataframe
Qval=pd.read_csv('qvalues.txt', sep=' ')
to_correct['qvalue']=list(Qval['qvalue'])
to_correct['significant']=list(Qval['significant'])
print(sum(to_correct['significant']))


KeyError: 'qvalue'

In [17]:
ensgene

NameError: name 'ensgene' is not defined

'STR_31054363'

In [ ]:


for ensgene in allgenes:
    count=count+1
    print (ensgene, 'Starting...')#,esnp_results[esnp_results["gene"]==ensgene][SNP_FDR_METHOD])
    estr_fdr = min(estr_results[estr_results["gene"]==ensgene][STR_FDR_METHOD])
    esnp_fdr = min(esnp_results[esnp_results["gene"]==ensgene][SNP_FDR_METHOD])
    #print(ensgene, estr_fdr.shape, esnp_fdr.shape)
    probes = expr_annot[expr_annot["gene.id"]==ensgene]["probe.id"].values
    if len(probes) != 1:
        PROGRESS("ERROR: no unique probe for %s"%ensgene, printit=DEBUG)
        continue
    else: probe = probes[0]
    
    if esnp_fdr==[] or estr_fdr==[]:
        PROGRESS("ERROR: qvalue for %s was not calculated for one these (SNPs or STRs)"%ensgene, printit=DEBUG)
        continue
    
    PROGRESS("Getting data for %s (%s)"%(ensgene, probe), printit=DEBUG)
    # Get eSTRs # get eSNPs
    estrs = estr_results[(estr_results["gene"]==ensgene)]
    esnps = esnp_results[(esnp_results["gene"]==ensgene)]
    esnps = esnps.sort_values(SNP_FDR_METHOD).head(10)  #Taking only top 10 snps
    # Get genotypes and expression values
    y = pd.DataFrame({"expr":list(expr.loc[:,probe])})
    y.index = str_samples
    cis_snps = snpgt[snpgt["start"].apply(lambda x: x in list(esnps["str.start"].values))]
    cis_strs = strgt[strgt["start"].apply(lambda x: x in list(estrs["str.start"].values))]
    print(cis_strs.shape, cis_snps.shape, '-------------cis variants---------------')
    # Run one analysis per STR
    for i in range(cis_strs.shape[0]):
        # Get STR and SNP data
        locus_str = cis_strs.iloc[[i],:][str_samples].transpose()
        locus_str.index = str_samples
        locus_str.columns = ["STR_%s"%(cis_strs["start"].values[i])]
        locus_snp = cis_snps[str_samples].transpose()
        print ("STR Locus ", i, '\t ', locus_str.shape, locus_snp.shape)
        if len(locus_snp.columns) == 0:
            PROGRESS("Skipping %s, no SNP data."%ensgene, printit=DEBUG)
            continue
        # Get subsets
        samples_to_keep = [str_samples[k] for k in range(len(str_samples)) \
                           if (str(locus_str.iloc[:,0].values[k]) != "None" \
                               and (str(locus_snp.iloc[:,0].values[k]) != "None"))]
        locus_str = locus_str.loc[samples_to_keep,:].replace('None', np.nan).astype(float)
        locus_snp = locus_snp.loc[samples_to_keep,:].replace('None', np.nan).astype(float)
        locus_y = y.loc[samples_to_keep,:]
        
        # Get data frame with relevant variables
        d = {"STR": ZNorm(locus_str.iloc[:,0].values), "expr": ZNorm(locus_y["expr"].values)}
        for k in range(locus_snp.shape[1]):
            snps = ZNorm(locus_snp.iloc[:,k].values)
            if snps is not None:
                d["SNP%s"%k] = snps
        genedata = pd.DataFrame(d)
        print(genedata.shape, '::::::::::::::::::::::::')
        # Run regression with SNPs
        if locus_snp.shape[1] > 0 and "SNP0" in genedata.columns:
            snpstring = "+".join([item for item in list(genedata.columns) if "SNP" in item])
            formula_snpstr = "expr ~ STR + " + snpstring
            formula_snp = "expr ~ " + snpstring
            formula_str = "expr ~ STR"
            lm_snpstr = ols(formula_snpstr, genedata).fit()
            lm_snp = ols(formula_snp, genedata).fit()
            lm_str = ols(formula_str, genedata).fit()
            bic_snpstr = lm_snpstr.bic
            bic_snp = lm_snp.bic
            aic_snpstr = lm_snpstr.aic
            aic_snp = lm_snp.aic
            snpstr_rsq = lm_snpstr.rsquared
            snp_rsq = lm_snp.rsquared
            str_rsq = lm_str.rsquared
            anova_results = anova_lm(lm_snp, lm_snpstr)
            pval = anova_results["Pr(>F)"].values[1]
            print (",".join(map(str, [CHROM, ensgene, cis_strs["start"].values[i], locus_snp.shape[1],locus_y.shape[0], \
                                           str_rsq, snp_rsq,snpstr_rsq, pval, estr_fdr, esnp_fdr,bic_snp-bic_snpstr, aic_snp-aic_snpstr])))
        else:
            formula_str = "expr ~ STR"
            lm_str = ols(formula_str, genedata).fit()
            str_rsq = lm_str.rsquared
            print (",".join(map(str, [CHROM, ensgene, cis_strs["start"].values[i], locus_snp.shape[1],\
                                          locus_y.shape[0], str_rsq, 0, str_rsq, -1, estr_fdr, -1, 0, 0])))
    